In [1]:
"""
IMPORTANT: Select the 'whisper' kernel in PyCharm for CUDA support!
Kernel: Python 3.12 (whisper)
"""
import openai
import whisper
import os
import glob
from tqdm import tqdm
import torch
from pathlib import Path

# Add ffmpeg to PATH for this session
os.environ['PATH'] = r'C:\ffmpeg\bin' + os.pathsep + os.environ.get('PATH', '')

# Check CUDA availability and set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")

Using device: cuda
CUDA Device: NVIDIA GeForce RTX 4090


In [2]:
def transcribe_audio_openai(audio_path, output_txt):
    """Use OpenAI API for transcription"""
    client = openai.OpenAI()

    with open(str(audio_path), "rb") as audio_file:
        response = client.audio.transcriptions.create(
            file=audio_file,
            model="whisper-1",   # Modèle officiel OpenAI
            language="en"        # Optionnel si la langue est sûre
        )

    with open(str(output_txt), "w", encoding="utf-8") as f:
        f.write(response.text)

def transcribe_audio(audio_path, output_txt):
    """Use local Whisper model with automatic device detection"""
    # Load model with device auto-detection and map_location fallback
    try:
        model = whisper.load_model("large-v3", device=device)
    except RuntimeError as e:
        if "CUDA" in str(e):
            print(f"Warning: CUDA model loading failed, falling back to CPU")
            model = whisper.load_model("large-v3", device="cpu")
        else:
            raise
    
    # Convert Path to string for whisper
    result = model.transcribe(str(audio_path), language="en")
    with open(str(output_txt), "w", encoding="utf-8") as f:
        f.write(result['text'])

In [3]:
files = glob.glob("D:/trading/Stockbee/Bullish Momentum Burst Guide/**/*.wav", recursive=True)
# files = glob.glob("D:/trading/Warrior Trading/Warrior Trading Courses/**/*.wav", recursive=True)

# Normalize paths for Windows
files = [os.path.normpath(file) for file in files]

# remove existing text files from the list
print(f"Found {len(files)} files")
files = [file for file in files if not os.path.exists(os.path.splitext(file)[0] + ".txt")]
print(f"Found {len(files)} files to process")
files

Found 50 files
Found 50 files to process


['D:\\trading\\Stockbee\\Bullish Momentum Burst Guide\\Section 1\\2022-09-12-mom-burst-01.wav',
 'D:\\trading\\Stockbee\\Bullish Momentum Burst Guide\\Section 1\\2022-09-12-mom-burst-02.wav',
 'D:\\trading\\Stockbee\\Bullish Momentum Burst Guide\\Section 1\\2022-09-12-mom-burst-03.wav',
 'D:\\trading\\Stockbee\\Bullish Momentum Burst Guide\\Section 1\\2022-09-12-mom-burst-04.wav',
 'D:\\trading\\Stockbee\\Bullish Momentum Burst Guide\\Section 1\\2022-09-12-mom-burst-05.wav',
 'D:\\trading\\Stockbee\\Bullish Momentum Burst Guide\\Section 1\\2022-09-12-mom-burst-06.wav',
 'D:\\trading\\Stockbee\\Bullish Momentum Burst Guide\\Section 1\\2022-09-12-mom-burst-07.wav',
 'D:\\trading\\Stockbee\\Bullish Momentum Burst Guide\\Section 1\\2022-09-12-mom-burst-08.wav',
 'D:\\trading\\Stockbee\\Bullish Momentum Burst Guide\\Section 1\\2022-09-12-mom-burst-09.wav',
 'D:\\trading\\Stockbee\\Bullish Momentum Burst Guide\\Section 1\\2022-09-12-mom-burst-10.wav',
 'D:\\trading\\Stockbee\\Bullish Momentu

In [ ]:
pbar = tqdm(files)
print(f"Processing {len(files)} files")
for file in pbar:
    # Update the progress bar description
    pbar.set_description(f"Processing {file}")
    # Define the paths
    audio_path = Path(file)
    output_txt = audio_path.with_suffix('.txt')
    transcribe_audio(audio_path, output_txt)

Processing D:\trading\Stockbee\Bullish Momentum Burst Guide\Section 1\2022-09-12-mom-burst-01.wav:   0%|          | 0/50 [00:00<?, ?it/s]

Processing 50 files


C:\Users\otrem\miniconda3\envs\whisper-env\Lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=